### Import Libraries

The first step is to import the libraries that we will need.

In [41]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [42]:
wallaroo.__version__

'2023.2.0rc4'

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [43]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

In [44]:
workspace_name = 'moto2'
pipeline_name = 'bikedaypipe'
model_name = 'bikedaymodel'

## Set the Workspace and Pipeline


In [45]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
# pipeline

### Upload Model

This model is a simple ARIMA(1,0,1) with no exogenous variables, hard-coded to forecast seven days out.

Note that this package is being specified as a `python` configuration.

In [46]:
model_file_name = 'forecast.py'

bike_day_model = wl.upload_model(model_name, model_file_name).configure(runtime="python")

### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.

In [47]:
# Set the deployment to allow for additional engines to run
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(1)
                        .memory("1Gi")
                        .build()
                    )

# pipeline = wl.build_pipeline('pipeline')
pipeline.add_model_step(bike_day_model).deploy(deployment_config = deploy_config)

name,bikedaypipe
created,2023-05-18 18:55:59.011989+00:00
last_updated,2023-05-18 20:11:28.264659+00:00
deployed,True
tags,
versions,"af2cf9d4-1861-4771-a3f0-5bbc5b139c64, 8bd2604a-ea82-4890-80d9-051c9f4e07bd, f1b8b382-5286-4c6f-b452-42c808f65801"
steps,bikedaymodel


In [48]:
model_file_name = 'forecast.py'
bike_day_model_two = wl.upload_model(model_name, model_file_name).configure(runtime="python")

In [49]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(2).cpus(1).memory("1Gi").build()
pipeline_two = wl.build_pipeline('pipeline-two')
pipeline_two.add_model_step(bike_day_model).deploy(deployment_config = deployment_config)

name,pipeline-two
created,2023-05-18 18:56:18.312478+00:00
last_updated,2023-05-18 20:11:41.608958+00:00
deployed,True
tags,
versions,"bbefb90a-70d3-4687-88e8-f44a3bc77500, 25be25be-2cc0-43a3-9539-5d28f4b7c5a8, d11e832b-4086-4e40-af65-948983ee4c2c, 064b7488-9e47-4865-8ff4-163884dc4e2b, e1afa399-7eb0-497f-a5db-fb025bdbabcd, 7bc28275-3bf9-4dc7-82c1-b4a412b90f6e, 2e269d94-a257-4fbd-a862-2cb7a3058468, ee4e9eac-67c0-4122-918a-eed5175b08ba"
steps,bikedaymodel


In [50]:
pipeline_two._deployment._url()

'https://doc-test.api.wallaroocommunity.ninja/v1/api/pipelines/infer/pipeline-two-54/pipeline-two'

In [51]:
pipeline._deployment._url()

'https://doc-test.api.wallaroocommunity.ninja/v1/api/pipelines/infer/bikedaypipe-57/bikedaypipe'

## run 
./streamer http://engine-lb.pipeline-two-257:29502/pipelines/pipeline-two 2 bikerental_moto_demo_python/testdata_dict.json

### Run Inference
Run a test inference to verify the pipeline is operational.

In [52]:
results = pipeline.infer_from_file('testdata_dict.json', data_format="custom-json")
display(results)
# results[0].data()

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [53]:
pipeline.undeploy()

name,bikedaypipe
created,2023-05-18 18:55:59.011989+00:00
last_updated,2023-05-18 20:11:28.264659+00:00
deployed,True
tags,
versions,"af2cf9d4-1861-4771-a3f0-5bbc5b139c64, 8bd2604a-ea82-4890-80d9-051c9f4e07bd, f1b8b382-5286-4c6f-b452-42c808f65801"
steps,bikedaymodel
